In [1]:
import re
import string
from collections import Counter

In [2]:
import pandas as pd
# import dovpanda

In [3]:
from tqdm import tqdm, tqdm_pandas

In [4]:
df = pd.read_excel('FullYakut-Matched.xlsx', index_col=[0])

In [5]:
df.head()

,name,desc,althix,wikidataID,primary_name,enwiki_title,alternative_names,country,country_code,wikidata_classes,...,toponym_arabic_other,toponym_translit,coord_lat,toponym_search,region_code,coord_lon,region_spelled,top_type_hom,mlat,mlon
0,آبار الأعراب,# جمع بئر. يقال في جمعها آبار وبئار وأبار: موض...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,آبج,# بفتح الهمزة وبعد الألف باء موحدة مفتوحة وجيم...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,آبر,# بفتح الهمزة وسكون الألف وضم الباء الموحدة ور...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,آبسكون,# بفتح الهمزة وسكون الألف وفتح الباء الموحدة و...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,آبسكون,Ābaskūn,37.02772,Abaskun,Daylam,54.04327,al-Daylam,towns,37.027720,54.043270
4,آبل,# بفتح الهمزة وبعد الألف باء مكسورة ولام:\n<br...,NaN,Q6530147,Abil,Abil,"آبل,Абил",Syria,SY,village,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.637222,36.676111


In [7]:
sf = pd.read_excel('keywords-defs.xlsx')

In [8]:
sf.fillna(0, inplace=True)

In [9]:
sf.PreWB = sf.PreWB.astype(bool)
sf.PostWB = sf.PostWB.astype(bool)

In [10]:
sf

,Level,Keyword,PreWB,PostWB
0,1,ثغر,False,False
1,1,ثغور,False,False
2,1,حد,True,True
3,1,الحد,True,True
4,1,حدود,False,False
5,1,جبهة,False,False
6,1,جبهات,False,False
7,1,تخم,False,False
8,1,تخوم,False,False
9,1,العواصم,False,False


In [11]:
WB = r'\W'

In [13]:
# Build regexes
keystrings = []
subpatterns = []
for level, group in sf.groupby('Level'):
    l = []
    for wd in group.itertuples():
        prefix = WB if wd.PreWB else ''
        postfix = WB if wd.PostWB else ''
        l.append(f'{prefix}{wd.Keyword.strip()}{postfix}')
    subpatterns.append(l)

In [14]:
keystrings = ['|'.join(x) for x in subpatterns]

In [15]:
k1 = re.compile(f'{keystrings[0]}')
k2 = re.compile(f'{keystrings[1]}')

In [19]:
buf = []

for r in tqdm(df.itertuples(), total=df.shape[0]):
    hi = k1.findall(r.desc)
    lo = k2.findall(r.desc)
    lm = 0
    if hi:
        lm = 1
    elif lo:
        lm = 2
    if not lm:
        continue
    kyws = hi + lo
    matched = ', '.join(x.strip() for x in kyws)
    buf.append([r.name, r.desc, lm, matched])
    
nf = pd.DataFrame(buf, columns=['name', 'text', 'level', 'keywords'])

100%|██████████| 12454/12454 [00:01<00:00, 10058.01it/s]


In [20]:
nf

,name,text,level,keywords
0,آبسكون,# بفتح الهمزة وسكون الألف وفتح الباء الموحدة و...,2,طبرستان
1,وآبل القمح,# قرية من نواحي بانياس من أعمال دمشق بين دمشق ...,2,الخزر
2,آبه,# بالباء الموحدة: قال أبو سعد: قال الحافظ أبو ...,2,"الري., أذربيجان, الري،, طبرستان"
3,آذرم,# هكذا ضبطه أبو سعد بألف بعد الهمزة، وفتح الذا...,1,ثغور
4,آذنة,# بكسر الذال المعجمة والنون: خيال من أخيلة حمى...,1,حدود
...,...,...,...,...
1486,يغنى,# بلفظ مضارع غنا: قرية من نواحي نخشب بما وراء ...,2,ما وراء النهر
1487,اليمن,# بالتحريك، قال الشرقي: إنما سميت اليمن لتيامن...,1,"حدود, حد, حدود, حد, حدود, حد"
1488,ينخوب,# بالفتح ثم السكون، وآخره باء موحدة:\n<br /># ...,1,حد
1489,ينشتة,# بفتح أوله وثانيه، وشين معجمة ساكنة، وتاء مثن...,2,الخزر


In [21]:
nf.iloc[30]

name                                                  الأخراب
text        # جمع خرب، بالضم، وهو منقطع الرمل.\n<br /># قا...
level                                                       1
keywords                                                 ثغور
Name: 30, dtype: object

In [22]:
print(nf.iloc[30].text)

# جمع خرب، بالضم، وهو منقطع الرمل.
<br /># قال ابن حبيب: الأخراب أقيرن حمر بين PageV01P119
<br /># السجا والثعل، وحولهما، وهي لبني الأضبط، وبني قوالة، فما يلي الثعل
<br />~~لبني قوالة بن أبي ربيعة، وما يلي السجا لبني الأضبط بن كلاب، وهما من
<br />~~أكرم مياه نجد، وأجمعه لبني كلاب. وسجا بعيدة القعر، عذبة الماء، والثعل
<br />~~أكثرهما ماء، وهو شروب، وأجلى هضاب ثلاث على مبدأة من الثعل، قال طهمان بن
<br />~~عمرو الكلابي:
<br /># لن تجد الأخراب أيمن من سجا % % إلى الثعل، إلا ألأم الناس عامرة
<br /># وروي أن عمر بن الخطاب، رضي الله عنه، قال للراشد بن عبد رب السلمي: لا
<br />~~تسكن الأخراب، فقال: ضيعتي لا بد لي منها، فقال: لكأني أنظر إليك تعي
<br />~~أمثال الذآنين حتى تموت، فكان كذلك.
<br /># وقيل: الأخراب في هذا الموضع اسم للثغور، وأخراب عزور موضع في شعر جميل
<br />~~حين قال:
<br /># حلفت برب الواقصات إلى منى، % % وما سلك الأخراب أخراب عزور



In [23]:
trs = str.maketrans('', '', string.punctuation)

def clean(text):
    text = re.sub(r'Page\w*', '', text)
    text = text.replace('<br />', '').replace('#','').replace('~','').replace('Milestone300','')
    return text

def rmpunc(text):
    return text.translate(trs)

In [26]:
for i in range(40,50):
    ct = clean(df.iloc[i].desc)
    print(i, ct)
#     print(i, ' '.join(ct.split(':',1)[1].split()[:4]))

40  بلد نسب إليه نوع من الثياب. وآم قرية من الجزيرة في شعر عدي.

41  يلتقي في الميم ساكنتان ثم دال مهملة مكسورة وياء ساكنة وزاي: من قرى
بخارا، ويقال بغير مد، وقد ذكرت في موضعها .

42  بضم الميم واللام: اسم أكبر مدينة بطبرستان في السهل، لأن طبرستان سهل
وجبل، وهي في الإقليم الرابع، وطولها سبع وسبعون درجة وثلث، وعرضها سبع
وثلاثون درجة ونصف وربع. وبين آمل وسارية ثمانية عشر فرسخا، وبين آمل
والرويان اثنا عشر فرسخا، وبين آمل وسالوس، وهي من جهة الجيلان، عشرون
فرسخا. وقد ذكرنا خبر فتحها بطبرستان، فأغنى. وبآمل تعمل السجادات
الطبرية، والبسط الحسان، وكان بها أول إسلام أهلها مسلحة في ألفي رجل، وقد
خرج منها كثير من العلماء، لكنهم قل ما ينسبون إلى غير طبرستان فيقال لهم
الطبري، منهم أبو جعفر محمد بن جرير الطبري صاحب التفسير والتاريخ
المشهور، أصله ومولده من آمل، ولذلك قال أبو بكر محمد بن العباس
الخوارزمي، وأصله من آمل أيضا، وكان يزعم أن أبا جعفر الطبري خاله:
 بآمل مولدي، وبنو جرير % % فأخوالي، ويحكي المرء خاله
 فها أنا رافضي عن تراث، % % وغيري رافضي عن كلالة
 وكذب لم يكن أبو جعفر، رحمه الله، رافضيا، وإ

In [27]:
seqs = []
for r in df.itertuples():
    c = clean(r.desc)
    if ':' not in c:
        seqs.append(' '.join(c.split()[:4]))
    else:
        seqs.append(' '.join(c.split(':',1)[1].split()[:4]))

In [28]:
seqs = [rmpunc(t) for t in seqs]

In [29]:
with open('arabic_sws.txt') as f:
    sws = f.readlines()
    

In [33]:
# seqs

In [30]:
sws = [s.strip() for s in sws] + ['اسم']

In [35]:
tgs = []
ct = 0
for s in seqs:
    l = s.split()
    if not l:
        tgs.append(None)
        continue
    h = l[0]
    if h in sws:
        tgs.append(s)
    else:
        tgs.append(h)

In [36]:
df['Type'] = tgs

In [38]:
df.to_excel('NEW_with_type.xlsx')